In [1]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB
import matplotlib.pyplot as plt
import string
import matplotlib
import seaborn as sns
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
# minimum amount of looks per step
def looks_per_step(step, looks_min, changes):
    model.addConstr(squares[:, :, :, step].sum() >= looks_min )
    if step +1 < s:
        model.addConstr(chg[:, :, :, step].sum() >= changes )

In [3]:
def create_base_model(n, m, s):
    
    # initialize aux variables
    for look in range(m):
        for i in range(n):
            for j in range(n):
                for step in range(s-1):
                    model.addGenConstrAnd(chg[look, i, j, step] , [squares[look, i, j, step] , squares[look, i, j, step+1] ])
    
    # each look can only appear once per frame
    for look in range(m):
        for figure in range(s):
            model.addConstr(squares[look, :, :, figure].sum() <= 1)
            # changing this constraint to == doubles the sol time ?

    # NQC 1 - only look one per row or column
    for step in range(s):
        for i in range(n):
            model.addConstr(squares[:, i, :, step].sum() <= 1)
            model.addConstr(squares[:, :, i, step].sum() <= 1)

    # NQC 2 - only one look per left and right diagonal
    for step in range(s):
        model.addConstrs(gp.quicksum(squares[m, i, j, step] for m in range(m) for i in range(n) for j in range(n) if i - j == k) <= 1 for k in range(-n + 1, n - 1))
        model.addConstrs(gp.quicksum(squares[m, i, j, step] for m in range(m) for i in range(n) for j in range(n) if i + j == k) <= 1 for k in range(2 * n - 1)) 

In [4]:
def is_only_onstage(look, entry, exit):
    for figure in range(s):
        if figure > exit or figure < entry:
            model.addConstr(squares[look, :, :, figure].sum() == 0)
        else:
            model.addConstr(squares[look, :, :, figure].sum() == 1)

In [5]:
def init_config(look, y, x):
    model.addConstr(squares[look, x, y, 0] == 1)

In [6]:
def quad(look, q1, q2, q3, q4):
    if q1 == 1:
        model.addConstr(gp.quicksum(squares[look, i, j, figure] for figure in range(s) for i in range(n-f) for j in range(n-f) ) >= 1 )
    if q1 == 0:
        model.addConstr(gp.quicksum(squares[look, i, j, figure] for figure in range(s) for i in range(n-f) for j in range(n-f) ) == 0 )
    
    if q2 == 1:
        model.addConstr(gp.quicksum(squares[look, i, j, figure] for figure in range(s) for i in range(f, n) for j in range(n-f) ) >= 1 )
    if q2 == 0:
        model.addConstr(gp.quicksum(squares[look, i, j, figure] for figure in range(s) for i in range(f, n) for j in range(n-f) ) == 0 )
    
    if q3 == 1:
        model.addConstr(gp.quicksum(squares[look, i, j, figure] for figure in range(s) for i in range(n-f) for j in range(f, n) ) >= 1 )
    if q3 == 0:
        model.addConstr(gp.quicksum(squares[look, i, j, figure] for figure in range(s) for i in range(n-f) for j in range(f, n) ) == 0 )
    
    if q4 == 1:
        model.addConstr(gp.quicksum(squares[look, i, j, figure] for figure in range(s) for i in range(f, n) for j in range(f, n) ) >= 1 )
    if q4 == 0:
        model.addConstr(gp.quicksum(squares[look, i, j, figure] for figure in range(s) for i in range(f, n) for j in range(f, n) ) == 0 )
    

In [35]:
n = 8 # size of board
m = 7 # number of looks
s = 10 # number of steps
f = n//2

model = gp.Model('layered_schedule')
model.params.LogToConsole = 1
squares = model.addMVar((m, n, n, s), vtype=GRB.BINARY, name="c")
chg = model.addMVar((m, n, n, s-1), vtype=GRB.BINARY, name="standing_still")
    
create_base_model(n, m, s)

In [36]:
# initial setup form last phase, LOOK / X l.to.r / Y (from top to bot)
#init_config(0, 0, 7-4)
#init_config(1, 6, 7-7)
init_config(0, 1, 7-7)
init_config(1, 5, 7-5)
model.addConstr(squares[5, 6, 4, s-1] == 1)
model.addConstr(squares[6, 5, 6, s-1] == 1)

<gurobi.Constr *Awaiting Model Update*>

In [37]:
is_only_onstage(0, 0, 0)
is_only_onstage(1, 0, 0)
is_only_onstage(2, 0, s)
is_only_onstage(3, 0, s)
is_only_onstage(4, 0, s)
is_only_onstage(5, s-1, s)
is_only_onstage(6, s-1, s)

In [38]:

for step in range(1,s-1):
    print(step)
    looks_per_step(step, 3, 2)
looks_per_step(0, 5, 2)
#looks_per_step(s, 5, 0)


1
2
3
4
5
6
7
8


In [39]:
#quad(0, 1, 1, 1, 1)
#quad(1, 1, 1, 1, 1)
quad(2, 1, 1, 1, 1)
quad(3, 1, 1, 1, 1)
quad(4, 1, 1, 1, 1)
#quad(5, 1, 1, 1, 1)

In [40]:
model.optimize()
print('Runtime:  '+str(model.Runtime))

instr_arr = np.zeros((s,n,n))
for look in range(m):
    for figure in range(s):
        for i in range(n):
            for j in range(n):
                if np.rint(squares.X[look,i,j,figure]) == 1:
                    #print(figure, i, j)
                    instr_arr[figure,i,j] = look+1 + 4

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 624 rows, 8512 columns and 36798 nonzeros
Model fingerprint: 0x92d597e2
Model has 4032 general constraints
Variable types: 0 continuous, 8512 integer (8512 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve added 2796 rows and 0 columns
Presolve removed 0 rows and 5338 columns
Presolve time: 0.05s
Presolved: 3420 rows, 3174 columns, 17877 nonzeros
Variable types: 0 continuous, 3174 integer (3174 binary)

Root relaxation: objective 0.000000e+00, 2692 iterations, 0.12 seconds (0.16 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       0.0000000    0.00000  0.00%     -  

In [41]:
#(np.argwhere(squares.X[5,:,:,:] == 1))

In [42]:
instr_arr

array([[[ 0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.],
        [ 0.,  0.,  0.,  0.,  0.,  6.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [23]:
np.count_nonzero(instr_arr[9,:,:])

3

In [26]:
(np.argwhere(squares.X[5,:,:,:] == 1))

array([], shape=(0, 3), dtype=int64)